In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import ml.generators.mip_generator as generator
import tensorflow as tf

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [5]:
TrainGen = generator.MipGenerator
ValGen = generator.MipGenerator
train_gen = TrainGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=False)
val_gen = ValGen(dims=(220, 220, 3),
                          batch_size=16,
                          augment_data=True,
                          extend_dims=False,
                          validation=True)

In [6]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint
from keras.optimizers import adam

In [7]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(2048, activation='relu')(x)
    predictions = Dense(nb_classes, activation='sigmoid')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [8]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(220, 220, 3))
for layer in base_model.layers:
    layer.trainable = False
finished_model = add_new_last_layer(base_model, 1)
finished_model.compile(optimizer=adam(lr=0.0001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
finished_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 220, 220, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 220, 220, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 220, 220, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 110, 110, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 110, 110, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 110, 110, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 55, 55, 128)       0         
__________

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [ ]:
mc_callback = ModelCheckpoint(filepath='tmp/vgg16_pretrained_weights.hdf5', verbose=1)
finished_model.fit_generator(
    generator=train_gen.generate(),
    steps_per_epoch=train_gen.get_steps_per_epoch(),
    validation_data=val_gen.generate(),
    validation_steps=val_gen.get_steps_per_epoch(),
    epochs=20,
    callbacks=[mc_callback],
    verbose=1,
    max_queue_size=1)

0
Epoch 1/20
Loaded entire batch.
(16, 220, 220, 3)
1
 1/87 [..............................] - ETA: 8:25 - loss: 0.7028 - acc: 0.4375Loaded entire batch.
(16, 220, 220, 3)
2
 2/87 [..............................] - ETA: 5:57 - loss: 0.6924 - acc: 0.5312Loaded entire batch.
(16, 220, 220, 3)
3
 3/87 [>.............................] - ETA: 5:57 - loss: 0.6955 - acc: 0.4375Loaded entire batch.
(16, 220, 220, 3)
4
 4/87 [>.............................] - ETA: 5:48 - loss: 0.7008 - acc: 0.3906Loaded entire batch.
(16, 220, 220, 3)
5
 5/87 [>.............................] - ETA: 5:44 - loss: 0.6991 - acc: 0.4250Loaded entire batch.
(16, 220, 220, 3)
6
 6/87 [=>............................] - ETA: 5:44 - loss: 0.6993 - acc: 0.4271Loaded entire batch.
(16, 220, 220, 3)
7
 7/87 [=>............................] - ETA: 5:35 - loss: 0.6963 - acc: 0.4732Loaded entire batch.
(16, 220, 220, 3)
8
 8/87 [=>............................] - ETA: 5:30 - loss: 0.6961 - acc: 0.4844Loaded entire batch.
(16, 2

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Loaded entire batch.
(16, 220, 220, 3)
13
13/87 [===>..........................] - ETA: 5:02 - loss: 0.7020 - acc: 0.4952Loaded entire batch.
(16, 220, 220, 3)
14
14/87 [===>..........................] - ETA: 4:57 - loss: 0.6928 - acc: 0.5179Loaded entire batch.
(16, 220, 220, 3)
15
15/87 [====>.........................] - ETA: 4:53 - loss: 0.6932 - acc: 0.5208Loaded entire batch.
(16, 220, 220, 3)
16
16/87 [====>.........................] - ETA: 4:49 - loss: 0.6922 - acc: 0.5234Loaded entire batch.
(16, 220, 220, 3)
17
17/87 [====>.........................] - ETA: 4:44 - loss: 0.6863 - acc: 0.5368Loaded entire batch.
(16, 220, 220, 3)
18
18/87 [=====>........................] - ETA: 4:40 - loss: 0.6811 - acc: 0.5486Loaded entire batch.
(16, 220, 220, 3)
19
19/87 [=====>........................] - ETA: 4:34 - loss: 0.6774 - acc: 0.5559Loaded entire batch.
(16, 220, 220, 3)
20
20/87 [=====>........................] - ETA: 4:29 - loss: 0.6751 - acc: 0.5594Loaded entire batch.
(16, 220, 2

56/87 [==================>...........] - ETA: 1:51 - loss: 0.6359 - acc: 0.6183Loaded entire batch.
(16, 220, 220, 3)
57
57/87 [==================>...........] - ETA: 1:47 - loss: 0.6348 - acc: 0.6206Loaded entire batch.
(16, 220, 220, 3)
58
58/87 [===================>..........] - ETA: 1:44 - loss: 0.6344 - acc: 0.6218Loaded entire batch.
(16, 220, 220, 3)
59
59/87 [===================>..........] - ETA: 1:40 - loss: 0.6342 - acc: 0.6208Loaded entire batch.
(16, 220, 220, 3)
60
60/87 [===================>..........] - ETA: 1:37 - loss: 0.6328 - acc: 0.6240Loaded entire batch.
(16, 220, 220, 3)
61
61/87 [====================>.........] - ETA: 1:33 - loss: 0.6347 - acc: 0.6209Loaded entire batch.
(16, 220, 220, 3)
62
62/87 [====================>.........] - ETA: 1:30 - loss: 0.6348 - acc: 0.6200Loaded entire batch.
(16, 220, 220, 3)
63
63/87 [====================>.........] - ETA: 1:26 - loss: 0.6339 - acc: 0.6200Loaded entire batch.
(16, 220, 220, 3)
64
64/87 [=====================>...